#### 1. 项目环境配置与全局参数
定义项目所需的依赖库、硬件设备配置、文件路径及超参数。
若不需要重新训练模型，可直接运行单元格1，2，3，4，7。

In [ ]:
import os
import sys
import shutil
import hashlib
import jso
import warnings
from pathlib import Path
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import timm

# 忽略非必要的警告信息
warnings.filterwarnings('ignore')

class Config:
    # --- 路径配置 (相对路径) ---
    BASE_DIR = Path.cwd()
    DATA_DIR = BASE_DIR / "data"              # 原始数据集目录
    BACKUP_DIR = BASE_DIR / "data_backup"     # 备份目录 (存放重复文件)
    SAVE_DIR = BASE_DIR / "checkpoints"       # 模型权重保存目录
    
    # --- 训练超参数 ---
    model_name = 'swinv2_tiny_window16_256'   # 选用模型架构
    num_classes = 4                           # 分类数量
    img_size = 256                            # 输入图像尺寸
    batch_size = 32                           # 批次大小
    epochs = 20                               # 训练轮次
    lr = 5e-5                                 # 初始学习率
    seed = 42                                 # 随机种子 (保证复现性)
    
    # --- 硬件配置 ---
    device = "cuda" if torch.cuda.is_available() else "cpu"
    num_workers = 4 if os.name != 'nt' else 0 # Windows下建议设为0

# 确保必要的目录存在
Config.SAVE_DIR.mkdir(exist_ok=True, parents=True)
print(f"运行环境: {Config.device} | 模型架构: {Config.model_name}")

运行环境: cuda | 模型架构: swinv2_tiny_window16_256


#### 2. 工具函数 (数据清洗与可视化)
包含数据MD5去重校验功能，以及训练过程的Loss/Accuracy曲线绘制函数。

In [3]:
def check_and_clean_data(root_dir, backup_dir):
    """
    通过计算MD5哈希值检测并移除重复图片，防止数据泄漏。
    """
    root_dir = Path(root_dir)
    backup_dir = Path(backup_dir)
    
    if not root_dir.exists():
        print(f"提示: 数据目录 {root_dir} 不存在，请检查路径。")
        return

    backup_dir.mkdir(exist_ok=True)
    unique_hashes = {}
    duplicates_count = 0
    
    print("进行数据完整性校验...")
    image_exts = {'.jpg', '.jpeg', '.png', '.bmp', '.tif'}
    file_list = [p for p in root_dir.rglob('*') if p.suffix.lower() in image_exts]
    
    for file_path in tqdm(file_list, desc="扫描文件", leave=False):
        try:
            with open(file_path, "rb") as f:
                file_hash = hashlib.md5(f.read()).hexdigest()
            
            if file_hash in unique_hashes:
                # 发现重复，移动到备份目录
                target_dir = backup_dir / file_path.parent.name
                target_dir.mkdir(parents=True, exist_ok=True)
                shutil.move(str(file_path), str(target_dir / file_path.name))
                duplicates_count += 1
            else:
                unique_hashes[file_hash] = file_path
        except Exception as e:
            print(f"读取错误: {file_path} - {e}")

    if duplicates_count > 0:
        print(f"移除了 {duplicates_count} 张重复图片至 {backup_dir}")
    else:
        print("数据校验通过: 未发现重复样本。")

def plot_history(history, save_path=None):
    """
    绘制训练过程中的 Loss 和 Accuracy 变化曲线。
    """
    epochs = range(1, len(history['train_loss']) + 1)
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 绘制 Loss
    axes[0].plot(epochs, history['train_loss'], 'b.-', label='Train')
    axes[0].plot(epochs, history['val_loss'], 'r.-', label='Val')
    axes[0].set_title('Loss Curve')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    # 绘制 Accuracy
    axes[1].plot(epochs, history['train_acc'], 'b.-', label='Train')
    axes[1].plot(epochs, history['val_acc'], 'r.-', label='Val')
    axes[1].set_title('Accuracy Curve')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].legend()
    axes[1].grid(True)
    
    # 绘制 Learning Rate
    axes[2].plot(epochs, history['lr'], 'g.-')
    axes[2].set_title('Learning Rate Schedule')
    axes[2].set_xlabel('Epochs')
    axes[2].set_yscale('log')
    axes[2].grid(True)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.show()

#### 3. 数据预处理与加载
构建Dataset与DataLoader。训练集采用强增强策略，验证/测试集仅进行标准化处理。数据集按 7.5:1.5:1.5 比例进行物理隔离划分。

In [4]:
def get_dataloaders(cfg):
    # 训练集数据增强 (Augmentation)，包括随机裁剪、颜色抖动、仿射变换和水平翻转
    train_tf = transforms.Compose([
        transforms.RandomResizedCrop(cfg.img_size, scale=(0.8, 1.0)), 
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.RandomAffine(degrees=10, translate=(0.05, 0.05)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # 验证/测试集预处理 (Standardization)
    val_tf = transforms.Compose([
        transforms.Resize((cfg.img_size, cfg.img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    # 加载数据集
    if not cfg.DATA_DIR.exists():
        raise FileNotFoundError(f"数据目录不存在: {cfg.DATA_DIR}")

    # 使用两个实例分别应用不同的Transform
    full_train_set = datasets.ImageFolder(cfg.DATA_DIR, transform=train_tf)
    full_val_set = datasets.ImageFolder(cfg.DATA_DIR, transform=val_tf)
    
    # 确定划分索引 (使用固定种子确保每次运行测试集一致)
    total_len = len(full_train_set)
    indices = torch.randperm(total_len, generator=torch.Generator().manual_seed(cfg.seed)).tolist()
    
    train_len = int(0.7 * total_len)
    val_len = int(0.15 * total_len)
    
    train_idx = indices[:train_len]
    val_idx = indices[train_len : train_len + val_len]
    test_idx = indices[train_len + val_len:]
    
    # 构建子集
    train_ds = Subset(full_train_set, train_idx)
    val_ds = Subset(full_val_set, val_idx)
    test_ds = Subset(full_val_set, test_idx) # 测试集使用标准预处理
    
    # 构建DataLoader
    loaders = {
        'train': DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, 
                          num_workers=cfg.num_workers, pin_memory=True),
        'val': DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, 
                        num_workers=cfg.num_workers, pin_memory=True),
        'test': DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False, 
                         num_workers=cfg.num_workers, pin_memory=True)
    }
    
    print(f"数据加载完成 | 训练集: {len(train_ds)}, 验证集: {len(val_ds)}, 测试集: {len(test_ds)}")
    return loaders, len(full_train_set.classes)

#### 4. 模型构建与训练核心逻辑
定义模型加载函数、单轮训练函数 (train_one_epoch) 和 评估函数 (evaluate)

In [5]:
def build_model(cfg):
    """加载预训练模型并修改分类头"""
    print(f"正在构建模型: {cfg.model_name}...")
    model = timm.create_model(
        cfg.model_name, 
        pretrained=True, 
        num_classes=cfg.num_classes
    )
    return model.to(cfg.device)

def train_one_epoch(model, loader, optimizer, criterion, scaler, cfg):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    loop = tqdm(loader, desc="[Train]", leave=False)
    for x, y in loop:
        x, y = x.to(cfg.device), y.to(cfg.device)
        
        optimizer.zero_grad()
        # 混合精度训练
        with torch.amp.autocast('cuda', enabled=(cfg.device=='cuda')):
            pred = model(x)
            loss = criterion(pred, y)
            
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0) # 梯度裁剪
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
        correct += (pred.argmax(1) == y).sum().item()
        total += y.size(0)
        
        loop.set_postfix(loss=loss.item())
        
    return total_loss / len(loader), 100 * correct / total

@torch.no_grad()
def evaluate(model, loader, criterion, cfg):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    
    for x, y in tqdm(loader, desc="[Eval]", leave=False):
        x, y = x.to(cfg.device), y.to(cfg.device)
        with torch.amp.autocast('cuda', enabled=(cfg.device=='cuda')):
            pred = model(x)
            loss = criterion(pred, y)
            
        total_loss += loss.item()
        correct += (pred.argmax(1) == y).sum().item()
        total += y.size(0)
        
    return total_loss / len(loader), 100 * correct / total

#### 5. 执行模型训练
测试时这个单元格可以跳过

In [ ]:
# 1. 数据准备
check_and_clean_data(Config.DATA_DIR, Config.BACKUP_DIR)
loaders, num_classes = get_dataloaders(Config)

# 2. 模型与优化器初始化
model = build_model(Config)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=Config.lr, weight_decay=0.05)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.epochs)
scaler = torch.amp.GradScaler('cuda', enabled=(Config.device=='cuda'))

# 3. 训练循环
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': [], 'lr': []}
best_acc = 0.0

print(f"\n开始训练 ({Config.epochs} epochs)...")

for epoch in range(1, Config.epochs + 1):
    curr_lr = optimizer.param_groups[0]['lr']
    
    # 训练与验证
    t_loss, t_acc = train_one_epoch(model, loaders['train'], optimizer, criterion, scaler, Config)
    v_loss, v_acc = evaluate(model, loaders['val'], criterion, Config)
    
    scheduler.step()
    
    # 记录日志
    history['train_loss'].append(t_loss)
    history['val_loss'].append(v_loss)
    history['train_acc'].append(t_acc)
    history['val_acc'].append(v_acc)
    history['lr'].append(curr_lr)
    
    # 保存最佳权重
    save_msg = ""
    if v_acc > best_acc:
        best_acc = v_acc
        torch.save(model.state_dict(), Config.SAVE_DIR / "best_model.pth")
        save_msg = f"最佳模型已保存 ({best_acc:.2f}%)"
        
    print(f"Epoch {epoch:02d} | LR: {curr_lr:.2e} | "
          f"Train: {t_loss:.4f} / {t_acc:.2f}% | "
          f"Val: {v_loss:.4f} / {v_acc:.2f}% {save_msg}")

print(f"\n训练结束，最高验证集准确率: {best_acc:.2f}%")

#### 6. 结果可视化
绘制并保存训练过程中的各项指标变化图。

In [ ]:
plot_history(history, save_path=Config.BASE_DIR / "training_result.png")
print(f"曲线图保存至: {Config.BASE_DIR / 'training_result.png'}")

#### 7. 模型评估与测试
本单元格将直接加载保存的最佳模型权重 (best_model.pth)，并在独立的测试集 (Test Set) 上进行推理，输出最终的评估指标。不需要重新进行训练。

In [8]:
def run_inference_test():
    weight_path = Config.SAVE_DIR / "best_model.pth"
    
    if not weight_path.exists():
        print(f"错误: 找不到权重文件 {weight_path}。请先运行上方训练代码或上传权重文件。")
        return

    print(f"正在加载最佳权重: {weight_path}")
    print("准备测试集数据...")
    
    # 重新获取Loader (保证测试集划分与训练时一致)
    loaders, _ = get_dataloaders(Config)
    test_loader = loaders['test']
    
    # 初始化模型结构
    model = build_model(Config)
    # 加载权重
    model.load_state_dict(torch.load(weight_path, map_location=Config.device))
    
    # 定义测试用的Loss
    criterion = nn.CrossEntropyLoss()
    
    print("\ninference...")
    test_loss, test_acc = evaluate(model, test_loader, criterion, Config)
    
    print(f"Test Accuracy: {test_acc:.2f}%")
    print(f"Test Loss:     {test_loss:.4f}")

# 执行推理
if __name__ == "__main__":
    run_inference_test()

正在加载最佳权重: d:\IT\CODE\JUPYTER\ConvNeXt_V2\checkpoints\best_model.pth
准备测试集数据...
数据加载完成 | 训练集: 4940, 验证集: 1058, 测试集: 1060
正在构建模型: swinv2_tiny_window16_256...

inference...


Test Accuracy: 90.75%
Test Loss:     0.2939
